In [78]:
import numpy as np
from scipy.linalg import circulant
import sympy

In [2]:
l_a = np.array([3/4, 1/4, -1/8, 0, -1/8, 1/4])
h_a = np.array([-1/2, 1, -1/2, 0 ,0, 0])
l_s = np.array([1, 1/2, 0, 0, 0, 1/2])
h_s = np.array([-1/4, -1/8, 0, -1/8, -1/4, 3/4])


In [5]:
print(f'l_a+l_s {l_a+l_s}')
print(f'h_a+h_s {h_a+h_s}')
print(f'l_a+h_a {l_a+h_a}')
print(f'l_s+h_s {l_s+h_s}')
print(f'l_a+h_s {l_a+h_s}')
print(f'l_s+h_a {l_s+h_a}')

l_a+l_s [ 1.75   0.75  -0.125  0.    -0.125  0.75 ]
h_a+h_s [-0.75   0.875 -0.5   -0.125 -0.25   0.75 ]
l_a+h_a [ 0.25   1.25  -0.625  0.    -0.125  0.25 ]
l_s+h_s [ 0.75   0.375  0.    -0.125 -0.25   1.25 ]
l_a+h_s [ 0.5    0.125 -0.125 -0.125 -0.375  1.   ]
l_s+h_a [ 0.5  1.5 -0.5  0.   0.   0.5]


In [7]:
print(f'sum(l_a) {np.sum(l_a)}')
print(f'sum(l_s) {np.sum(l_s)}')
print(f'sum(h_a) {np.sum(h_a)}')
print(f'sum(h_s) {np.sum(h_s)}')


sum(l_a) 1.0
sum(l_s) 2.0
sum(h_a) 0.0
sum(h_s) 0.0


In [8]:
def conv_circular(x, h, verbose=False):
    N = len(x)
    M = len(h)
    w = np.zeros(N)
    for k in range(N):
        if verbose: print(f'w[{k}] = ', end=' ')
        for m in range(M):
            if verbose: print(f'h[{m}] * x[{k-m}] ({h[m]}*{x[k-m]}={h[m] * x[k-m]})', end= '')
            if verbose and m+1 < M: print(' + ', end='')
            w[k] += h[m] * x[k-m]
        if verbose: print(f' = {w[k]}')
    return w

In [11]:
x = np.array([1, -2, 2, 4, 8, 3])

In [13]:
def one_stage_analysis(x, l_analysis, h_analysis): 
    N = len(x)
   
    Xl = conv_circular(x,l_analysis)[::2]
    Xh = conv_circular(x,h_analysis)[::2]
    
    X = np.concatenate([Xl, Xh])
    return X, Xl, Xh

X, Xl, Xh = one_stage_analysis(x, l_a, h_a)
X

array([-0.25 ,  0.875,  7.375, -1.5  , -3.5  , -1.   ])

array([ 1., -2.,  2.,  4.,  8.,  3.])

In [26]:
print(l_a)
print()
print(circulant(l_a))
print()
print(circulant(l_a)[::2])

[ 0.75   0.25  -0.125  0.    -0.125  0.25 ]

[[ 0.75   0.25  -0.125  0.    -0.125  0.25 ]
 [ 0.25   0.75   0.25  -0.125  0.    -0.125]
 [-0.125  0.25   0.75   0.25  -0.125  0.   ]
 [ 0.    -0.125  0.25   0.75   0.25  -0.125]
 [-0.125  0.    -0.125  0.25   0.75   0.25 ]
 [ 0.25  -0.125  0.    -0.125  0.25   0.75 ]]

[[ 0.75   0.25  -0.125  0.    -0.125  0.25 ]
 [-0.125  0.25   0.75   0.25  -0.125  0.   ]
 [-0.125  0.    -0.125  0.25   0.75   0.25 ]]


In [76]:
def my_circulant(filter_vector):
    assert type(filter_vector) == np.ndarray, 'Apenas por sanidade'
    N = len(filter_vector)
    res = np.zeros((N,N))
    for row in range(N):
        res[row] = np.roll(filter_vector, row)
    return res.T # pensava que que rolava na linhas, mas na verdade rola nas colunas
 
def test_circulant(sample):
    golden = circulant(sample)
    dut = my_circulant(sample)
    
    assert np.allclose(golden, dut), 'Teste falhou'
    print('Teste passou')

test_circulant(l_a)
test_circulant(l_s)
test_circulant(h_a)
test_circulant(h_s)

Teste passou
Teste passou
Teste passou
Teste passou


In [63]:
def matrizDWT_analysis(l_analysis, h_analysis):
    l_analysis_circulant = circulant(l_analysis)
    h_analysis_circulant = circulant(h_analysis)

    l_analysis_circulant_downsampled = l_analysis_circulant[::2]
    h_analysis_circulant_downsampled = h_analysis_circulant[::2]

    W = np.concatenate([l_analysis_circulant_downsampled, h_analysis_circulant_downsampled])
    return W

matrizDWT_analysis(l_a, h_a)

array([[ 0.75 ,  0.25 , -0.125,  0.   , -0.125,  0.25 ],
       [-0.125,  0.25 ,  0.75 ,  0.25 , -0.125,  0.   ],
       [-0.125,  0.   , -0.125,  0.25 ,  0.75 ,  0.25 ],
       [-0.5  ,  0.   ,  0.   ,  0.   , -0.5  ,  1.   ],
       [-0.5  ,  1.   , -0.5  ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.5  ,  1.   , -0.5  ,  0.   ]])

In [58]:
print(circulant(l_a))
print()
print(my_circulant(l_a))
print(np.allclose(circulant(l_a),my_circulant(l_a)))

[[ 0.75   0.25  -0.125  0.    -0.125  0.25 ]
 [ 0.25   0.75   0.25  -0.125  0.    -0.125]
 [-0.125  0.25   0.75   0.25  -0.125  0.   ]
 [ 0.    -0.125  0.25   0.75   0.25  -0.125]
 [-0.125  0.    -0.125  0.25   0.75   0.25 ]
 [ 0.25  -0.125  0.    -0.125  0.25   0.75 ]]

[[ 0.75   0.25  -0.125  0.    -0.125  0.25 ]
 [ 0.25   0.75   0.25  -0.125  0.    -0.125]
 [-0.125  0.25   0.75   0.25  -0.125  0.   ]
 [ 0.    -0.125  0.25   0.75   0.25  -0.125]
 [-0.125  0.    -0.125  0.25   0.75   0.25 ]
 [ 0.25  -0.125  0.    -0.125  0.25   0.75 ]]
True


In [65]:
def matrizDWT_synthesis(l_synthesis, h_synthesis):
    l_synthesis_circulant = circulant(l_synthesis)
    h_synthesis_circulant = circulant(h_synthesis)

    l_analysis_circulant_upnsampled = l_synthesis_circulant[::2]
    h_analysis_circulant_upsampled = h_synthesis_circulant[::2]

    W = np.concatenate([l_analysis_circulant_upnsampled, h_analysis_circulant_upsampled], axis=1)
    return W

matrizDWT_analysis(l_a, h_a)

array([[ 0.75 ,  0.25 , -0.125,  0.   , -0.125,  0.25 ],
       [-0.125,  0.25 ,  0.75 ,  0.25 , -0.125,  0.   ],
       [-0.125,  0.   , -0.125,  0.25 ,  0.75 ,  0.25 ],
       [-0.5  ,  0.   ,  0.   ,  0.   , -0.5  ,  1.   ],
       [-0.5  ,  1.   , -0.5  ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.5  ,  1.   , -0.5  ,  0.   ]])

In [67]:
np.round(np.linalg.inv(matrizDWT_analysis(l_a, h_a)),2)

array([[ 1.  ,  0.  ,  0.  , -0.25, -0.25,  0.  ],
       [ 0.5 ,  0.5 ,  0.  , -0.12,  0.75, -0.12],
       [ 0.  ,  1.  ,  0.  ,  0.  , -0.25, -0.25],
       [ 0.  ,  0.5 ,  0.5 , -0.12, -0.12,  0.75],
       [ 0.  ,  0.  ,  1.  , -0.25, -0.  , -0.25],
       [ 0.5 ,  0.  ,  0.5 ,  0.75, -0.12, -0.13]])

In [85]:
sympy.Matrix(my_circulant(np.array([np.sqrt(2)/2, np.sqrt(2)/2,0 ,0]))[::2])

Matrix([
[0.707106781186548,               0.0,               0.0, 0.707106781186548],
[              0.0, 0.707106781186548, 0.707106781186548,               0.0]])

In [86]:
sympy.Matrix(my_circulant(np.array([np.sqrt(2)/2, -np.sqrt(2)/2,0 ,0]))[::2])

Matrix([
[0.707106781186548,                0.0,               0.0, -0.707106781186548],
[              0.0, -0.707106781186548, 0.707106781186548,                0.0]])

Matrix([[0.707106781186548, 0.707106781186548]])

In [135]:
def matrix_downsample(circulant_matrix):
    M, N = circulant_matrix.shape
    assert M==N, 'Matriz circulant deve ser quadrada'
    assert M%2==0,  'Dimensões devem ser múltiplo de 2'
    
    matrix = np.zeros((M//2,N), dtype='int')
    for i in range (M//2):
        for j in range(N):
            if j == 2*i:
                matrix[i][j] = 1
                
    return matrix

c_tmp = circulant(np.array([np.sqrt(2)/2, np.sqrt(2)/2,0 ,0]))
d_tmp = matrix_downsample(c_tmp)
Xl_tmp = np.dot(d_tmp, c_tmp)

In [112]:
def matrix_upsample(downsampled_signal):
    M, N = downsampled_signal.shape
    assert M==N//2, 'Signal que foi downsampled'
    assert M%2==0,  'Dimensões devem ser múltiplo de 2'    
    
    matrix = np.zeros((M,N))
    for i in range (M):
        for j in range(N):
            if 2*j == i:
                matrix[i][j] = 1    
    
    print(matrix)
    return matrix

u_tmp = matrix_downsample(c_tmp).T
np.dot(Xl_tmp, u_tmp)

array([[0.70710678, 0.        ],
       [0.        , 0.70710678]])

In [114]:
Xl_tmp.shape

(2, 4)

In [115]:
l_a

array([ 0.75 ,  0.25 , -0.125,  0.   , -0.125,  0.25 ])

In [141]:
from sympy import Matrix, Rational
l_a_s = Matrix([
    [Rational(3,4), Rational(1/4), -Rational(1/8), 0, -Rational(1/8), Rational(1,4)],
    [Rational(1,4), Rational(3,4), Rational(1/4), -Rational(1/8), 0, -Rational(1/8)],
    [-Rational(1/8), Rational(1,4), Rational(3,4), Rational(1/4), -Rational(1/8), 0],
    [0, -Rational(1/8), Rational(1,4), Rational(3,4), Rational(1/4), -Rational(1/8)],
    [-Rational(1/8), 0, -Rational(1/8), Rational(1,4), Rational(3,4), Rational(1/4)],
    [Rational(1/4), -Rational(1/8), 0, -Rational(1/8), Rational(1,4), Rational(3,4)]

]).T
l_a_s

Matrix([
[ 3/4,  1/4, -1/8,    0, -1/8,  1/4],
[ 1/4,  3/4,  1/4, -1/8,    0, -1/8],
[-1/8,  1/4,  3/4,  1/4, -1/8,    0],
[   0, -1/8,  1/4,  3/4,  1/4, -1/8],
[-1/8,    0, -1/8,  1/4,  3/4,  1/4],
[ 1/4, -1/8,    0, -1/8,  1/4,  3/4]])

In [125]:
circulant(l_a)

array([[ 0.75 ,  0.25 , -0.125,  0.   , -0.125,  0.25 ],
       [ 0.25 ,  0.75 ,  0.25 , -0.125,  0.   , -0.125],
       [-0.125,  0.25 ,  0.75 ,  0.25 , -0.125,  0.   ],
       [ 0.   , -0.125,  0.25 ,  0.75 ,  0.25 , -0.125],
       [-0.125,  0.   , -0.125,  0.25 ,  0.75 ,  0.25 ],
       [ 0.25 , -0.125,  0.   , -0.125,  0.25 ,  0.75 ]])

In [136]:
d_tmp = matrix_downsample(circulant(l_a))
d_tmp

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]])

In [137]:
d_sym = Matrix(d_tmp)
d_sym


Matrix([
[1, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 1, 0]])

In [138]:
d_sym * l_a_s

Matrix([
[ 3/4, 1/4, -1/8,   0, -1/8, 1/4],
[-1/8, 1/4,  3/4, 1/4, -1/8,   0],
[-1/8,   0, -1/8, 1/4,  3/4, 1/4]])

In [139]:
circulant(h_a)

array([[-0.5,  0. ,  0. ,  0. , -0.5,  1. ],
       [ 1. , -0.5,  0. ,  0. ,  0. , -0.5],
       [-0.5,  1. , -0.5,  0. ,  0. ,  0. ],
       [ 0. , -0.5,  1. , -0.5,  0. ,  0. ],
       [ 0. ,  0. , -0.5,  1. , -0.5,  0. ],
       [ 0. ,  0. ,  0. , -0.5,  1. , -0.5]])

In [143]:
h_a_s = Matrix([
    [-Rational(1,2), 1, -Rational(1,2), 0, 0, 0],
    [0,-Rational(1,2), 1, -Rational(1,2), 0, 0],
    [0,0,-Rational(1,2), 1, -Rational(1,2), 0],
    [0,0,0,-Rational(1,2), 1, -Rational(1,2)],
    [-Rational(1,2),0,0,0,-Rational(1,2), 1],
    [1,-Rational(1,2),0,0,0,-Rational(1,2)],
]).T
h_a_s

Matrix([
[-1/2,    0,    0,    0, -1/2,    1],
[   1, -1/2,    0,    0,    0, -1/2],
[-1/2,    1, -1/2,    0,    0,    0],
[   0, -1/2,    1, -1/2,    0,    0],
[   0,    0, -1/2,    1, -1/2,    0],
[   0,    0,    0, -1/2,    1, -1/2]])

In [144]:
d_sym * h_a_s

Matrix([
[-1/2, 0,    0, 0, -1/2, 1],
[-1/2, 1, -1/2, 0,    0, 0],
[   0, 0, -1/2, 1, -1/2, 0]])

In [145]:
matrizDWT_analysis(l_a, h_a)

array([[ 0.75 ,  0.25 , -0.125,  0.   , -0.125,  0.25 ],
       [-0.125,  0.25 ,  0.75 ,  0.25 , -0.125,  0.   ],
       [-0.125,  0.   , -0.125,  0.25 ,  0.75 ,  0.25 ],
       [-0.5  ,  0.   ,  0.   ,  0.   , -0.5  ,  1.   ],
       [-0.5  ,  1.   , -0.5  ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.5  ,  1.   , -0.5  ,  0.   ]])

In [150]:
Wa_6_sym = Matrix([
    [d_sym * l_a_s], 
    [d_sym * h_a_s]
])
Wa_6_sym

Matrix([
[ 3/4, 1/4, -1/8,   0, -1/8, 1/4],
[-1/8, 1/4,  3/4, 1/4, -1/8,   0],
[-1/8,   0, -1/8, 1/4,  3/4, 1/4],
[-1/2,   0,    0,   0, -1/2,   1],
[-1/2,   1, -1/2,   0,    0,   0],
[   0,   0, -1/2,   1, -1/2,   0]])

In [151]:
U = d_sym.T
U

Matrix([
[1, 0, 0],
[0, 0, 0],
[0, 1, 0],
[0, 0, 0],
[0, 0, 1],
[0, 0, 0]])

In [154]:
circulant(l_s)

array([[1. , 0.5, 0. , 0. , 0. , 0.5],
       [0.5, 1. , 0.5, 0. , 0. , 0. ],
       [0. , 0.5, 1. , 0.5, 0. , 0. ],
       [0. , 0. , 0.5, 1. , 0.5, 0. ],
       [0. , 0. , 0. , 0.5, 1. , 0.5],
       [0.5, 0. , 0. , 0. , 0.5, 1. ]])

In [156]:
l_s_s = Matrix([
    [1, Rational(1,2),0,0,0,Rational(1/2)],
    [Rational(1/2),1, Rational(1,2),0,0,0],
    [0,Rational(1/2),1, Rational(1,2),0,0],
    [0,0,Rational(1/2),1, Rational(1,2),0],
    [0,0,0,Rational(1/2),1, Rational(1,2)],
    [Rational(1,2),0,0,0,Rational(1/2),1],
]).T
l_s_s

Matrix([
[  1, 1/2,   0,   0,   0, 1/2],
[1/2,   1, 1/2,   0,   0,   0],
[  0, 1/2,   1, 1/2,   0,   0],
[  0,   0, 1/2,   1, 1/2,   0],
[  0,   0,   0, 1/2,   1, 1/2],
[1/2,   0,   0,   0, 1/2,   1]])

In [157]:
circulant(h_s)

array([[-0.25 ,  0.75 , -0.25 , -0.125,  0.   , -0.125],
       [-0.125, -0.25 ,  0.75 , -0.25 , -0.125,  0.   ],
       [ 0.   , -0.125, -0.25 ,  0.75 , -0.25 , -0.125],
       [-0.125,  0.   , -0.125, -0.25 ,  0.75 , -0.25 ],
       [-0.25 , -0.125,  0.   , -0.125, -0.25 ,  0.75 ],
       [ 0.75 , -0.25 , -0.125,  0.   , -0.125, -0.25 ]])

In [159]:
h_s_s = Matrix([
    [-Rational(1,4),-Rational(1,8),0,-Rational(1,8),-Rational(1,4),Rational(3,4)],
    [Rational(3,4),-Rational(1,4),-Rational(1,8),0,-Rational(1,8),-Rational(1,4)],
    [-Rational(1,4),Rational(3,4),-Rational(1,4),-Rational(1,8),0,-Rational(1,8)],
    [-Rational(1,8),-Rational(1,4),Rational(3,4),-Rational(1,4),-Rational(1,8),0],
    [0,-Rational(1,8),-Rational(1,4),Rational(3,4),-Rational(1,4),-Rational(1,8)],
    [-Rational(1,8),0,-Rational(1,8),-Rational(1,4),Rational(3,4),-Rational(1,4)],
]).T
h_s_s

Matrix([
[-1/4,  3/4, -1/4, -1/8,    0, -1/8],
[-1/8, -1/4,  3/4, -1/4, -1/8,    0],
[   0, -1/8, -1/4,  3/4, -1/4, -1/8],
[-1/8,    0, -1/8, -1/4,  3/4, -1/4],
[-1/4, -1/8,    0, -1/8, -1/4,  3/4],
[ 3/4, -1/4, -1/8,    0, -1/8, -1/4]])

In [161]:
l_s_s*U

Matrix([
[  1,   0,   0],
[1/2, 1/2,   0],
[  0,   1,   0],
[  0, 1/2, 1/2],
[  0,   0,   1],
[1/2,   0, 1/2]])

In [162]:
h_s_s*U

Matrix([
[-1/4, -1/4,    0],
[-1/8,  3/4, -1/8],
[   0, -1/4, -1/4],
[-1/8, -1/8,  3/4],
[-1/4,    0, -1/4],
[ 3/4, -1/8, -1/8]])

In [166]:
Ws_sym = (l_s_s*U).row_join(h_s_s*U)
Ws_sym

Matrix([
[  1,   0,   0, -1/4, -1/4,    0],
[1/2, 1/2,   0, -1/8,  3/4, -1/8],
[  0,   1,   0,    0, -1/4, -1/4],
[  0, 1/2, 1/2, -1/8, -1/8,  3/4],
[  0,   0,   1, -1/4,    0, -1/4],
[1/2,   0, 1/2,  3/4, -1/8, -1/8]])

In [168]:
Wa_6_sym*Ws_sym

Matrix([
[1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 1]])

In [170]:
Wa_6_sym

Matrix([
[ 3/4, 1/4, -1/8,   0, -1/8, 1/4],
[-1/8, 1/4,  3/4, 1/4, -1/8,   0],
[-1/8,   0, -1/8, 1/4,  3/4, 1/4],
[-1/2,   0,    0,   0, -1/2,   1],
[-1/2,   1, -1/2,   0,    0,   0],
[   0,   0, -1/2,   1, -1/2,   0]])

In [176]:
Ws_sym

Matrix([
[  1,   0,   0, -1/4, -1/4,    0],
[1/2, 1/2,   0, -1/8,  3/4, -1/8],
[  0,   1,   0,    0, -1/4, -1/4],
[  0, 1/2, 1/2, -1/8, -1/8,  3/4],
[  0,   0,   1, -1/4,    0, -1/4],
[1/2,   0, 1/2,  3/4, -1/8, -1/8]])

In [177]:
Wa_6_sym.row(0)

Matrix([[3/4, 1/4, -1/8, 0, -1/8, 1/4]])

In [178]:
Ws_sym.col(0)

Matrix([
[  1],
[1/2],
[  0],
[  0],
[  0],
[1/2]])

In [179]:
Wa_6_sym.row(0)* Ws_sym.col(0)

Matrix([[1]])

In [180]:
Wa_6_sym.row(1)

Matrix([[-1/8, 1/4, 3/4, 1/4, -1/8, 0]])

In [181]:
Ws_sym.col(1)

Matrix([
[  0],
[1/2],
[  1],
[1/2],
[  0],
[  0]])

In [182]:
Wa_6_sym.row(1)*Ws_sym.col(1)

Matrix([[1]])

In [183]:
Wa_6_sym.row(0)

Matrix([[3/4, 1/4, -1/8, 0, -1/8, 1/4]])

In [185]:
Ws_sym.col(1)

Matrix([
[  0],
[1/2],
[  1],
[1/2],
[  0],
[  0]])

In [186]:
Wa_6_sym.row(0)*Ws_sym.col(1)

Matrix([[0]])

In [187]:
Wa_6_sym.row(0)

Matrix([[3/4, 1/4, -1/8, 0, -1/8, 1/4]])

In [188]:
Ws_sym.col(2)

Matrix([
[  0],
[  0],
[  0],
[1/2],
[  1],
[1/2]])

In [189]:
Wa_6_sym.row(0)*Ws_sym.col(2)

Matrix([[0]])